Вычисление барицентрических координат

In [ ]:
from core import barisentrik_coordinates

_v = 1, 2
_v0 = 0.5, 0.5
_v1 = 1.0, 3.5
_v2 = 2.5, 0.7

lambda0, lambda1, lambda2 = barisentrik_coordinates(_v, _v0, _v1, _v2)

print(f'lambda0 + lambda1 + lambda2 = {lambda0 + lambda1 + lambda2}')

lambda0 + lambda1 + lambda2 = 0.9999999999999999


Отрисовка треугольников

In [ ]:
from core import CustomImage, draw_face
from names import WHITE

_v0 = -5.5, 5.5
_v1 = 25.5, 5.5
_v2 = 10, 25.5
img = CustomImage(20, 20)
draw_face((_v0, _v1, _v2), img, WHITE)
img.save('pictures/triangles/triangle_1.png')

_v0 = 5.5, 5.5
_v1 = 14.5, 5.5
_v2 = 10, 14.5
img = CustomImage(20, 20)
draw_face((_v0, _v1, _v2), img, WHITE)
img.save('pictures/triangles/triangle_2.png')

_v0 = 5.5, 5.5
_v1 = 16.5, 5.5
_v2 = 16.5, 25.5
img = CustomImage(20, 20)
draw_face((_v0, _v1, _v2), img, WHITE)
img.save('pictures/triangles/triangle_3.png')

Отрисовка полигонов трехмерной модели

In [ ]:
from core import CustomImage, ObjModel, draw_face
from names import FOX, DEER
from typing import Callable
from type import ObjVertex
from random import randint


def draw_visible_faces(obj: ObjModel, _img: CustomImage, transform: Callable[[ObjVertex], ObjVertex]):
    for v0, v1, v2 in obj.faces():
        v0 = transform(v0)
        v1 = transform(v1)
        v2 = transform(v2)

        x0, y0, z0 = v0
        x1, y1, z1 = v1
        x2, y2, z2 = v2

        v0 = x0, y0
        v1 = x1, y1
        v2 = x2, y2

        color = (randint(0, 255), randint(0, 255), randint(0, 255))

        draw_face((v0, v1, v2), img, color)


fox_obj = ObjModel(FOX)
deer_obj = ObjModel(DEER)

img = CustomImage(1000, 1000)
draw_visible_faces(fox_obj, img, lambda v: (4 * v[2] + 500, 4 * -v[1] + 500, 4 * v[0]))
img.save('pictures/faces/fox_rainbow.png')

img = CustomImage(1500, 1500)
draw_visible_faces(deer_obj, img, lambda v: (v[0] + 800, -v[1] + 1500, v[2]))
img.save('pictures/faces/deer_rainbow.png')

Отсечение нелицевых граней

In [ ]:
from core import CustomImage, ObjModel, draw_face, face_angle_cos
from names import FOX, DEER
from typing import Callable
from type import ObjVertex
from random import randint


def draw_visible_faces(obj: ObjModel, _img: CustomImage, transform: Callable[[ObjVertex], ObjVertex]):
    for f in obj.faces():
        v0, v1, v2 = f

        v0 = transform(v0)
        v1 = transform(v1)
        v2 = transform(v2)

        cos = face_angle_cos(f)
        if cos >= 0:
            continue

        x0, y0, z0 = v0
        x1, y1, z1 = v1
        x2, y2, z2 = v2

        v0 = x0, y0
        v1 = x1, y1
        v2 = x2, y2

        f = v0, v1, v2

        color = (randint(0, 255), randint(0, 255), randint(0, 255))

        draw_face(f, img, color)


fox_obj = ObjModel(FOX)
deer_obj = ObjModel(DEER)

img = CustomImage(1000, 1000)
draw_visible_faces(fox_obj, img, lambda v: (4 * v[2] + 500, 4 * -v[1] + 500, 4 * v[0]))
img.save('pictures/faces/fox_rainbow_ext.png')

img = CustomImage(1500, 1500)
draw_visible_faces(deer_obj, img, lambda v: (v[0] + 800, -v[1] + 1500, v[2]))
img.save('pictures/faces/deer_rainbow_ext.png')

Базовое освещение

In [ ]:
from core import CustomImage, ObjModel, draw_face, face_angle_cos
from names import FOX, DEER
from typing import Callable
from type import ObjVertex


def draw_visible_faces(obj: ObjModel, _img: CustomImage, transform: Callable[[ObjVertex], ObjVertex]):
    for f in obj.faces():
        v0, v1, v2 = f

        v0 = transform(v0)
        v1 = transform(v1)
        v2 = transform(v2)

        f = v0, v1, v2

        cos = face_angle_cos(f)
        if cos >= 0:
            continue

        x0, y0, z0 = v0
        x1, y1, z1 = v1
        x2, y2, z2 = v2

        v0 = x0, y0
        v1 = x1, y1
        v2 = x2, y2

        f = v0, v1, v2

        val = round(255 * -cos)
        color = (val, val, val)

        draw_face(f, img, color)


fox_obj = ObjModel(FOX)
deer_obj = ObjModel(DEER)

img = CustomImage(1000, 1000)
draw_visible_faces(fox_obj, img, lambda v: (4 * v[2] + 500, 4 * -v[1] + 500, 4 * v[0]))
img.save('pictures/faces/fox_basic_lighting.png')

img = CustomImage(1500, 1500)
draw_visible_faces(deer_obj, img, lambda v: (v[0] + 800, -v[1] + 1500, v[2]))
img.save('pictures/faces/deer_basic_lighting.png')

z-буфер

In [ ]:
from sys import float_info
from core import CustomImage, ObjModel, face_angle_cos
from names import FOX, DEER
from typing import Callable
from type import ObjVertex


def draw_visible_faces(obj: ObjModel, _img: CustomImage, transform: Callable[[ObjVertex], ObjVertex]):
    z_buffer = [[float_info.max for _ in range(_img.width())] for _ in range(_img.height())]
    for f in obj.faces():
        v0, v1, v2 = f

        v0 = transform(v0)
        v1 = transform(v1)
        v2 = transform(v2)

        f = v0, v1, v2

        cos = face_angle_cos(f)
        if cos > 0:
            continue

        x0, y0, z0 = v0
        x1, y1, z1 = v1
        x2, y2, z2 = v2

        v0 = x0, y0
        v1 = x1, y1
        v2 = x2, y2

        val = round(255 * -cos)
        color = (val, val, val)

        x_min = round(max(min(x0, x1, x2), 0))
        y_min = round(max(min(y0, y1, y2), 0))
        x_max = round(min(max(x0, x1, x2), _img.width() - 1))
        y_max = round(min(max(y0, y1, y2), _img.height() - 1))

        for i in range(x_min, x_max + 1):
            for j in range(y_min, y_max + 1):
                v = (i, j)
                l0, l1, l2 = barisentrik_coordinates(v, v0, v1, v2)
                if l0 > 0 and l1 > 0 and l2 > 0:
                    z = l0 * z0 + l1 * z1 + l2 * z2
                    if z < z_buffer[i][j]:
                        _img.set(v, color)
                        z_buffer[i][j] = z


fox_obj = ObjModel(FOX)
deer_obj = ObjModel(DEER)

img = CustomImage(1000, 1000)
draw_visible_faces(fox_obj, img, lambda v: (4 * v[2] + 500, 4 * -v[1] + 500, 4 * v[0] + 500))
img.save('pictures/faces/fox_z_buffer.png')

img = CustomImage(1500, 1500)
draw_visible_faces(deer_obj, img, lambda v: (v[0] + 800, -v[1] + 1500, v[2] + 1000))
img.save('pictures/faces/deer_z_buffer.png')